# Model experiments

Единый ноутбук для экспериментов с моделями.

Правило: **никаких предсказаний/подбора на test**. Сравнение и подбор — только через CV на train.

Финальный holdout test используется только один раз после выбора пайплайна.

In [2]:
import sys
from pathlib import Path

# Добавляем корень проекта в sys.path, чтобы работал `import src...`
ROOT = Path().resolve().parent if Path().resolve().name == "notebooks" else Path().resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [4]:
import pandas as pd
from IPython.display import Markdown, display

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from src.utils import (
    load_clean_df,
    make_feature_sets,
    make_features_info_df,
    build_pipeline_for_training_vova,
    evaluate_cv,
)

# Варианты фичей — как в 05_Feature_Engineering.ipynb
df = load_clean_df(column_names=None)
datasets = make_feature_sets(df)

X_with_body, _y = datasets["with_body"]
X_no_body, _ = datasets["no_body"]

display(Markdown("#### `with_body`"))
display(make_features_info_df(X_with_body))
display(Markdown("#### `no_body`"))
display(make_features_info_df(X_no_body))


#### `with_body`

,group_ru,feature,description_ru,dtype
0,Признаки пищевых привычек,CAEC,Перекусы между приемами пищи,category
1,Признаки пищевых привычек,CALC,Употребление алкоголя,category
2,Признаки пищевых привычек,CH2O,Потребление воды в день,category
3,Признаки пищевых привычек,FAVC,Частое употребление высококалорийной пищи,category
4,Признаки пищевых привычек,FCVC,Частота употребления овощей,category
5,Признаки пищевых привычек,NCP,Количество основных приемов пищи,category
6,Признаки физического состояния,FAF,Частота физической активности,category
7,Признаки физического состояния,MTRANS,Используемый транспорт,category
8,Признаки физического состояния,SCC,Самоконтроль потребления калорий,category
9,Признаки физического состояния,TUE,Время использования электронных устройств,category


#### `no_body`

,group_ru,feature,description_ru,dtype
0,Признаки пищевых привычек,CAEC,Перекусы между приемами пищи,category
1,Признаки пищевых привычек,CALC,Употребление алкоголя,category
2,Признаки пищевых привычек,CH2O,Потребление воды в день,category
3,Признаки пищевых привычек,FAVC,Частое употребление высококалорийной пищи,category
4,Признаки пищевых привычек,FCVC,Частота употребления овощей,category
5,Признаки пищевых привычек,NCP,Количество основных приемов пищи,category
6,Признаки физического состояния,FAF,Частота физической активности,category
7,Признаки физического состояния,MTRANS,Используемый транспорт,category
8,Признаки физического состояния,SCC,Самоконтроль потребления калорий,category
9,Признаки физического состояния,TUE,Время использования электронных устройств,category


## 1) Генерируем 6 вариантов датасета и сохраняем в `data/processed/`

In [5]:
display(Markdown("### Размерность вариантов"))
rows = []
for variant, (X, y) in datasets.items():
    rows.append({"variant": variant, "n_rows": len(X), "n_features": X.shape[1], "n_classes": y.nunique()})
pd.DataFrame(rows).sort_values("variant").reset_index(drop=True)

### Размерность вариантов

,variant,n_rows,n_features,n_classes
0,no_body,2087,14,4
1,with_body,2087,16,4


## 2) Быстрое сравнение моделей через CV (f1_macro)

In [6]:
import pandas as pd

models = {
    "logreg": LogisticRegression(max_iter=3000),
    "rf": RandomForestClassifier(n_estimators=600, random_state=42, n_jobs=-1),
    "svm_rbf": SVC(kernel="rbf"),
}

rows = []
for variant, (X, y) in datasets.items():
    for name, m in models.items():
        pipe = build_pipeline_for_training_vova(X, model=m, onehot_nominal=True, scale_numeric=True)
        metrics = evaluate_cv(pipe, X, y, cv_splits=5, seed=42)
        rows.append({"variant": variant, "model": name, **metrics})

pd.DataFrame(rows).sort_values(["variant", "f1_macro"], ascending=[True, False])

,variant,model,f1_macro,accuracy,cv_splits,seed
4,no_body,rf,0.792141,0.834220,5,42
5,no_body,svm_rbf,0.678393,0.733594,5,42
3,no_body,logreg,0.523237,0.618593,5,42
1,with_body,rf,0.934867,0.953048,5,42
0,with_body,logreg,0.928013,0.952573,5,42
2,with_body,svm_rbf,0.927247,0.949217,5,42


## 3) Подбор гиперпараметров для логистической регрессии (penalty/C)

Это самое важное из текущих задач.

In [7]:
from IPython.display import Markdown, display
import pandas as pd

display(Markdown("## 3) GridSearchCV: отдельно `with_body` и `no_body`"))

param_grid = {
    "model__C": [0.01, 0.1, 1, 3, 10],
    "model__penalty": ["l2"],
    "model__solver": ["lbfgs", "saga"],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rows = []
for variant in ["with_body", "no_body"]:
    X, y = datasets[variant]
    pipe = build_pipeline_for_training_vova(
        X,
        model=LogisticRegression(max_iter=5000),
        onehot_nominal=True,
        scale_numeric=True,
    )

    gs = GridSearchCV(pipe, param_grid=param_grid, scoring="f1_macro", cv=cv, n_jobs=-1)
    gs.fit(X, y)

    rows.append(
        {
            "variant": variant,
            "n_features": int(X.shape[1]),
            "best_score_f1_macro": float(gs.best_score_),
            **gs.best_params_,
        }
    )

pd.DataFrame(rows).sort_values("best_score_f1_macro", ascending=False).reset_index(drop=True)


## 3) GridSearchCV: отдельно `with_body` и `no_body`

/usr/local/Caskroom/miniconda/base/envs/obesity-risk/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/obesity-risk/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/obesity-risk/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1135: Future

,variant,n_features,best_score_f1_macro,model__C,model__penalty,model__solver
0,with_body,16,0.960085,10,l2,lbfgs
1,no_body,14,0.523366,3,l2,saga


## 4) Полиномиальная логистическая регрессия

Если нужно: добавляем полиномиальные признаки (может переобучаться).

In [8]:
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.pipeline import Pipeline
from IPython.display import Markdown, display
import pandas as pd

display(Markdown("## 4) PolynomialFeatures + LogReg: отдельно `with_body` и `no_body`"))

to_dense = FunctionTransformer(
    lambda a: a.toarray() if hasattr(a, "toarray") else a,
    accept_sparse=True,
)

rows = []
for variant in ["with_body", "no_body"]:
    X, y = datasets[variant]
    pre_pipe = build_pipeline_for_training_vova(
        X,
        model=LogisticRegression(max_iter=8000),
        onehot_nominal=True,
        scale_numeric=True,
    )

    poly_logreg = Pipeline(steps=[
        ("preprocess", pre_pipe.named_steps["preprocess"]),
        ("to_dense", to_dense),
        ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        ("model", LogisticRegression(max_iter=8000)),
    ])

    metrics = evaluate_cv(poly_logreg, X, y, cv_splits=5, seed=42)
    rows.append({"variant": variant, "n_features": int(X.shape[1]), **metrics})

pd.DataFrame(rows).sort_values("f1_macro", ascending=False).reset_index(drop=True)


## 4) PolynomialFeatures + LogReg: отдельно `with_body` и `no_body`

,variant,n_features,f1_macro,accuracy,cv_splits,seed
0,with_body,16,0.949962,0.963114,5,42
1,no_body,14,0.699557,0.745095,5,42
